In [ ]:
!pip install transformers
!pip install datasets
!mkdir ./checkpoints

In [23]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

UsageError: Line magic function `%` not found.


In [19]:
TRAIN_DATA_PATH = "https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/main/datasets/WASSA23_essay_level_train_preproc.tsv"
df_train = pd.read_csv(TRAIN_DATA_PATH, sep='\t')
df_train.head(10)

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,essay_id,emotion,Surprise,Hope,Neutral,Sadness,Joy,Fear,Anger,Disgust
0,2,35,It breaks my heart to see people living in tho...,6.833333,6.625,30,1,6,3,37,...,1,Hope/Sadness,0,1,0,1,0,0,0,0
1,3,35,I wonder why there aren't more people trying t...,5.833333,6.000,19,1,6,2,32,...,2,Anger,0,0,0,0,0,0,1,0
2,5,35,"After reading the article, you can't help but ...",1.000000,1.375,17,1,6,1,29,...,4,Sadness,0,0,0,1,0,0,0,0
3,6,213,It is so sad that someone who had such an amaz...,6.166667,6.625,16,2,5,1,28,...,5,Sadness,0,0,0,1,0,0,0,0
4,8,213,"From reading the article, it looks like the wo...",6.833333,1.000,30,1,6,3,37,...,7,Neutral,0,0,1,0,0,0,0,0
5,10,213,That's sad. Regardless of what they find out ...,1.666667,1.125,49,1,5,1,31,...,9,Sadness,0,0,0,1,0,0,0,0
6,11,78,"After reading the article, my reaction is that...",1.500000,1.000,17,1,6,1,29,...,10,Sadness,0,0,0,1,0,0,0,0
7,13,78,It sounds like these boys had a really rough l...,2.000000,1.000,24,2,7,1,38,...,12,Sadness,0,0,0,1,0,0,0,0
8,14,78,This is a tragic and sad story about how some ...,6.000000,3.000,43,2,6,1,33,...,13,Sadness,0,0,0,1,0,0,0,0
9,17,336,Hello. I feel really terrible about the curren...,7.000000,1.000,31,unknown,unknown,unknown,unknown,...,16,Disgust/Sadness,0,0,0,1,0,0,0,1


In [4]:
DEV_DATA_PATH = "https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/main/datasets/WASSA23_essay_level_dev_preproc.tsv"
df_dev = pd.read_csv(DEV_DATA_PATH, sep='\t')
df_dev.head(10)

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,...,iri_fantasy,iri_empathatic_concern,Sadness,Anger,Surprise,Neutral,Joy,Hope,Disgust,Fear
0,1,35,How sad is it that this kind of pain and suffe...,68,2,2,1,21,20000,1,...,3.143,3.286,1,0,0,0,0,0,0,0
1,4,35,The article is kind of tragic and hits close t...,79,1,6,3,33,64000,1,...,2.429,1.429,1,0,0,0,0,0,0,0
2,7,213,"I think that these kinds of stories, are sad, ...",68,2,2,1,21,20000,1,...,3.143,3.286,1,0,0,0,0,0,0,0
3,9,213,It's crazy that random accidents like this hap...,84,2,4,1,25,55000,1,...,3.571,3.143,0,0,0,1,0,0,0,0
4,12,78,This story makes me so so sad.... As someone w...,68,2,2,1,21,20000,1,...,3.143,3.286,1,0,0,0,0,0,0,0
5,15,78,"After reading the article, my first reaction a...",70,1,6,1,29,85000,1,...,4.143,4.643,1,0,0,0,0,0,0,0
6,16,336,I didn't know coal mining had such adverse eff...,81,1,4,1,30,27000,1,...,4.571,4.000,0,0,0,1,0,0,0,0
7,20,336,This is very sad. I can't imagine having elep...,73,2,7,1,38,42000,1,...,2.571,3.857,1,0,0,0,0,0,0,0
8,23,281,"Guys, reading this article really hits home fo...",63,1,4,1,25,29000,1,...,2.571,4.857,1,0,0,0,0,0,0,0
9,26,171,Hey guys. So I just read this article about Ir...,63,1,4,1,25,29000,1,...,2.571,4.857,0,0,0,1,0,0,0,0


In [20]:
class EmotionsDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len, target_labels):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.essay = dataframe.essay
        self.targets = self.data[target_labels].values
        self.max_len = max_len # TODO: ??

    def __len__(self):
        return len(self.essay)

    def __getitem__(self, index):
        essay = str(self.essay[index])
        essay = " ".join(essay.split()) # replace whitespace(s) with single space (TODO: check! move to preprocessing?)

        inputs = self.tokenizer.encode_plus(
            essay,
            None, # TODO: CHECK!
            add_special_tokens=True, # Adds '[CLS]' and '[SEP]'
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
# Function to calculate the accuracy
def flat_accuracy(preds, labels): # TODO: ha senso?
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [21]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01
OPT_STEP_SIZE = 1
OPT_GAMMA = 0.9
EPOCHS = 2
RANDOM_STATE = 42 # TODO: fix it everywhere?


tokenizer = BertTokenizer.from_pretrained('bert-base-cased', truncation=True)
emotions = ["Sadness", "Anger", "Surprise", "Neutral",	"Joy", "Hope", "Disgust", "Fear"]

train_size = 0.8
df_train = df_train[:10] ######################## TEST SUBSAMPLE ########################
train_dataset = df_train.sample(frac=train_size,random_state=RANDOM_STATE)
val_dataset = df_train.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df_train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

training_set = EmotionsDataset(dataframe=train_dataset, tokenizer=tokenizer, max_len=MAX_LEN, target_labels=emotions)
val_set = EmotionsDataset(dataframe=val_dataset, tokenizer=tokenizer, max_len=MAX_LEN, target_labels=emotions)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True, # TODO: SEED?
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
eval_dataloader = DataLoader(val_set, **val_params)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(emotions))

num_training_steps = EPOCHS * len(training_loader) # TODO: num_training_steps è okay?
progress_bar = tqdm(range(num_training_steps))
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("======= CUDA Available =======")
else:
    device = torch.device('cpu')
    print("======= CUDA NOT Available, run on CPU =======")
model.to(device)

training_stats = []
best_val_loss = float("inf")

for epoch in range(EPOCHS):
    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch+1, EPOCHS))
    print('Training...')

    total_train_loss = 0

    model.train()
    for i, batch in enumerate(training_loader):
        print('  Batch {:>5,}  of  {:>5,}'.format(i+1, len(training_loader))) # TODO: numero batch non torna
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['targets'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        total_train_loss += outputs.loss.item()
        optimizer.zero_grad()
        outputs.loss.backward() # TODO: che loss viene usata?
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
        progress_bar.update(1)

    avg_train_loss = total_train_loss / len(training_loader)
    
    print("")
    print("Average training loss: {0:.2f}".format(avg_train_loss))

    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Validation...")

    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in eval_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['targets'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        total_eval_loss += outputs.loss.item()
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    
    avg_val_accuracy = total_eval_accuracy / len(eval_dataloader)
    print("Validation Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(eval_dataloader)
    print("Validation Loss: {0:.2f}".format(avg_val_loss))

    if avg_val_loss < best_val_loss:
        print("Saving checkpoint!")
        best_val_loss = avg_val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': best_val_loss,
            },
            f"checkpoints/model.pt"
        )

    training_stats.append(
        {
            'epoch': epoch + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy
        }
    )

FULL Dataset: (792, 32)
TRAIN Dataset: (634, 32)
TEST Dataset: (158, 32)


KeyboardInterrupt: 

In [ ]:
# Create a DataFrame from training statistics
pd.set_option('display.precision', 2)
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

In [ ]:
# Plot the learning curve.
sns.set(style='darkgrid')
plt.rcParams["figure.figsize"] = (12,6)

plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks(np.arange(1, EPOCHS + 1, 1.0))

plt.show()

In [ ]:
# TODO: come accedere al modello salvato
# TODO: perchè da il warning dei pesi?
# TODO: perchè Adam è deprecato?
# TODO: testare il modello allenato, usare metriche multiclasse e scrivere gli output nel formato richiesto così da usare script evaluate
# TODO: cercare parametri di defualt con cui provarlo e farlo girare su tutti i dati
# TODO: riscrivere in modo modulare, così che basti cambiare il tokenizer e il nome del modello (provare il classificatore con strati sopra? provare a concatenare features?)
# TODO: salva i risultati in csv, salva curva
# TODO: vedi baselines altri (quali parametri hanno usato)